In [1]:
import cv2
import torch
import time
import numpy as np
import time
import datetime
from time import gmtime, strftime

In [2]:
model_type = "MiDaS_small"

'''

For higher accuracy/slower time:

model_type = "DPT_Hybrid"

For highest accuracy/slowest time:

model_type = "DPT_Large"

'''

midas = torch.hub.load("intel-isl/MiDaS", model_type)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
midas.to(device)
midas.eval()

Using cache found in /Users/alifakhry/.cache/torch/hub/intel-isl_MiDaS_master


Loading weights:  None


Using cache found in /Users/alifakhry/.cache/torch/hub/rwightman_gen-efficientnet-pytorch_master


MidasNet_small(
  (pretrained): Module(
    (layer1): Sequential(
      (0): Conv2dSameExport(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
      (1): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
      (3): Sequential(
        (0): DepthwiseSeparableConv(
          (conv_dw): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (bn1): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
          (act1): ReLU6(inplace=True)
          (se): Identity()
          (conv_pw): Conv2d(32, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn2): BatchNorm2d(24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
          (act2): Identity()
        )
      )
      (4): Sequential(
        (0): InvertedResidual(
          (conv_pw): Conv2d(24, 144, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(144,

In [4]:
midas_transforms = torch.hub.load("intel-isl/MiDaS", "transforms")

Using cache found in /Users/alifakhry/.cache/torch/hub/intel-isl_MiDaS_master


In [ ]:
def print_info(depth_map):
    img_reverted= cv2.bitwise_not(depth_map)
    new_img = img_reverted / 255.0 
    vertical_half = len(new_img)//2
    horizontal_half = len(new_img[0])//2
    
    print(new_img[0][horizontal_half])
    print(new_img[0][0])
    print(new_img[vertical_half][len(new_img[0])-1])
    print(new_img[vertical_half][horizontal_half])
    print(new_img[vertical_half][0])

In [ ]:
if model_type == "DPT_Large" or model_type == "DPT_Hybrid":
    transform = midas_transforms.dpt_transform
else:
    transform = midas_transforms.small_transform
    
cap = cv2.VideoCapture(0)
    
while cap.isOpened():
    success, img = cap.read()
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    input_batch = transform(img).to(device)
    
    with torch.no_grad():
        
        prediction = midas(input_batch)
        
        prediction = torch.nn.functional.interpolate(
            prediction.unsqueeze(1),
            size = img.shape[:2],
            mode = "bicubic",
            align_corners = False,
        ).squeeze()
        
    depth_map = prediction.cpu().numpy()

    depth_map = cv2.normalize(depth_map, None, 0, 1, norm_type = cv2.NORM_MINMAX, dtype=cv2.CV_64F)
    
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        
    depth_map = (depth_map * 255).astype(np.uint8)
    depth_map = cv2.applyColorMap(depth_map, cv2.COLORMAP_BONE)

    cv2.putText(img, f'TIME (UTC): {str(strftime("%Y-%m-%d %H:%M:%S", gmtime()))}', (20,70), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0,255,0), 2)
    cv2.imshow('Image', img)
    cv2.imshow('Depth Map', depth_map)
    
    if cv2.waitKey(5) & 0xFF == 27:
        break
        
    depth_map = cv2.cvtColor(depth_map, cv2.COLOR_BGR2GRAY)
    
    print_info(depth_map)
    
cap.release()